In [49]:
import numpy as np
import pandas as pd

In [53]:
# import libraries
import quandl
import matplotlib.pyplot as plt
# put your api key
quandl.ApiConfig.api_key = 'vH-1ocJiGzdqi5byHY4k'
# load the data
df = quandl.get("BITFINEX/BTCJPY")

In [54]:
df

,High,Low,Mid,Last,Bid,Ask,Volume
Date,,,,,,,
2018-03-29,7.994363e+05,7.342434e+05,769080.0,7.706760e+05,769070.0,769090.0,11.655926
2018-03-30,7.710971e+05,6.955769e+05,719800.0,7.187839e+05,719790.0,719810.0,288.789982
2018-03-31,7.661500e+05,7.234000e+05,737820.0,7.388100e+05,737810.0,737830.0,38.338285
2018-04-01,7.451446e+05,6.837212e+05,722880.0,7.226900e+05,722640.0,723120.0,76.712521
2018-04-02,7.552932e+05,7.212312e+05,743760.0,7.416444e+05,743750.0,743770.0,54.832632
...,...,...,...,...,...,...,...
2021-03-20,6.535400e+06,6.295900e+06,6366450.0,6.363600e+06,6366300.0,6366600.0,199.009028
2021-03-21,6.381200e+06,6.034392e+06,6278500.0,6.274700e+06,6277800.0,6279200.0,302.515215
2021-03-22,6.358900e+06,5.845600e+06,5917250.0,5.917600e+06,5916900.0,5917600.0,242.728951


In [55]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
df_min_max = pd.DataFrame(mm.fit_transform(df),
                          index=df.index, columns=df.columns)

In [56]:
df_min_max

,High,Low,Mid,Last,Bid,Ask,Volume
Date,,,,,,,
2018-03-29,0.066627,0.060611,0.063032,0.063248,0.063033,0.063032,0.000486
2018-03-30,0.062162,0.054268,0.055216,0.055017,0.055216,0.055216,0.061378
2018-03-31,0.061383,0.058832,0.058074,0.058194,0.058074,0.058074,0.006349
2018-04-01,0.058073,0.052324,0.055704,0.055637,0.055668,0.055741,0.014780
2018-04-02,0.059672,0.058476,0.059016,0.058643,0.059017,0.059016,0.009973
...,...,...,...,...,...,...,...
2021-03-20,0.970348,0.972854,0.950816,0.950369,0.950800,0.950832,0.041652
2021-03-21,0.946054,0.929961,0.936866,0.936269,0.936763,0.936970,0.064394
2021-03-22,0.942540,0.898994,0.879569,0.879627,0.879521,0.879618,0.051258


In [57]:
from sklearn.model_selection import train_test_split

In [58]:
interval = 6
def make_data(data):
    x = []
    y = []
    last = list(data["Last"])
    for i in range(len(last)):
        if i < interval: continue
        y.append(last[i])
        xa = []
        for p in range(interval):
            d = i + p - interval
            xb = []
            for feature in data:
                features = data[feature]
                xb.append(features[d])
            xa.append(xb)
        x.append(xa)
    return (x, y)

In [62]:
train_df, test_df = train_test_split(df_min_max, test_size=0.2, shuffle=False)

In [66]:
train_x, train_y = make_data(train_df)
test_x, test_y = make_data(test_df)

In [67]:
train_x = np.array(train_x)
test_x = np.array(test_x)
print(train_x.shape)
print(test_x.shape)

(846, 6, 7)
(207, 6, 7)


In [68]:
train_y = np.array(train_y)
test_y = np.array(test_y)
print(train_y.shape)
print(test_y.shape)

(846,)
(207,)


In [106]:
batch_size=1
learning_rate=0.001
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from keras import optimizers
lstm_model = Sequential()
lstm_model.add(LSTM(100, batch_input_shape=(batch_size, 6, 7),
                    dropout=0.0, recurrent_dropout=0.0, stateful=True, return_sequences=True,
                    kernel_initializer='random_uniform'))
lstm_model.add(Dropout(0.4))
lstm_model.add(LSTM(60, dropout=0.0))
lstm_model.add(Dropout(0.4))
lstm_model.add(Dense(20,activation='relu'))
lstm_model.add(Dense(1,activation='sigmoid'))
optimizer = optimizers.RMSprop(lr=learning_rate)
lstm_model.compile(loss='mean_squared_error', optimizer=optimizer)

In [107]:
history = lstm_model.fit(train_x, train_y, epochs=10, verbose=2, batch_size=batch_size,
                        shuffle=False, validation_data=(test_x, test_y))

Epoch 1/10
846/846 - 6s - loss: 0.0023 - val_loss: 0.1377
Epoch 2/10
846/846 - 4s - loss: 3.9980e-04 - val_loss: 0.1337
Epoch 3/10
846/846 - 4s - loss: 3.5347e-04 - val_loss: 0.1304
Epoch 4/10
846/846 - 3s - loss: 3.0049e-04 - val_loss: 0.1317
Epoch 5/10
846/846 - 4s - loss: 7.2490e-04 - val_loss: 0.1347
Epoch 6/10
846/846 - 4s - loss: 1.6253e-04 - val_loss: 0.1339
Epoch 7/10
846/846 - 4s - loss: 1.5397e-04 - val_loss: 0.1337
Epoch 8/10
846/846 - 3s - loss: 9.7749e-05 - val_loss: 0.1392
Epoch 9/10
846/846 - 4s - loss: 8.8189e-05 - val_loss: 0.1363
Epoch 10/10
846/846 - 3s - loss: 9.1279e-05 - val_loss: 0.1387
